# Анализ базы данных крупного сервиса для чтения книг по подписке

Задача: проанализировать БД, чтобы помочь коллегам сформулировать ценностное предложение (выгоды предложения, объяснение почему клиенту будет хорошо, если он воспользуется продуктом)

In [1]:
# Импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# Задаем параметры подключения к БД:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

# Формируем строку соединения к БД:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], db_config['pwd'], db_config['host'], 
                                                         db_config['port'], db_config['db'])

# Подключаемся к БД:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Создадим функцию для вывода рез-ов:
def result(query):
    return pd.io.sql.read_sql(query, con = engine)

# Формируем sql-запросы:
books = ''' 
           SELECT * 
           FROM books
        '''

authors = ''' 
             SELECT * 
             FROM authors
          '''

publishers = ''' 
                SELECT * 
                FROM publishers
             '''

ratings = ''' 
             SELECT * 
             FROM ratings
          '''

reviews = ''' 
             SELECT * 
             FROM reviews
          '''

In [2]:
# Исследуем books:
books = result(books)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [3]:
# Исследуем authors:
authors = result(authors)
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [4]:
# Исследуем publishers:
publishers = result(publishers)
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [5]:
# Исследуем ratings:
ratings = result(ratings)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [6]:
# Исследуем reviews:
reviews = result(reviews)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


Отлично! Пропусков нет, типы данных на своих местах! Приступаем к заданиям!

* Посчитайте, сколько книг вышло после 1 января 2000 года;

In [7]:
count_books = '''
                 SELECT
                     COUNT(*)
                 FROM
                     books
                 WHERE
                     publication_date > '2000-01-01'
              '''
result(count_books)

,count
0,819


Чуть больше 80% современной литературы. Молодёжь будет довольна.

* Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [8]:
count_reviews = '''
                   SELECT
                        books.title AS title,
                        books.book_id AS book_id,
                        COUNT(reviews.review_id) AS cnt
                   FROM
                        books
                   INNER JOIN reviews ON reviews.book_id = books.book_id
                   GROUP BY books.book_id
                   ORDER BY cnt DESC
                   LIMIT 10
                '''
result(count_reviews)

,title,book_id,cnt
0,Twilight (Twilight #1),948,7
1,The Alchemist,627,6
2,The Da Vinci Code (Robert Langdon #2),696,6
3,The Glass Castle,734,6
4,The Hobbit or There and Back Again,750,6
5,The Road,854,6
6,Outlander (Outlander #1),497,6
7,The Curious Incident of the Dog in the Night-Time,695,6
8,Water for Elephants,963,6
9,Eat Pray Love,207,6


In [9]:
avg_rating = '''
                SELECT
                    books.title AS title,
                    books.book_id AS book_id,
                    ROUND(AVG(ratings.rating), 2) AS avg_rating
                FROM
                    books
                INNER JOIN ratings ON ratings.book_id = books.book_id 
                GROUP BY books.book_id
                ORDER BY avg_rating DESC
                LIMIT 10
             '''
result(avg_rating)

,title,book_id,avg_rating
0,The Ghost Map: The Story of London's Most Terr...,732,5.0
1,Evening Class,224,5.0
2,Wherever You Go There You Are: Mindfulness Me...,972,5.0
3,Neil Gaiman's Neverwhere,469,5.0
4,March,418,5.0
5,Light in August,390,5.0
6,The Demon-Haunted World: Science as a Candle i...,699,5.0
7,Act of Treason (Mitch Rapp #9),57,5.0
8,Dead Souls,182,5.0
9,Welcome to Temptation (Dempseys #1),967,5.0


In [10]:
count_reviews_and_avg_rating = '''
                                 SELECT
                                     SUBQ0.title,
                                     SUBQ0.avg_rating,
                                     SUBQ1.cnt
                                 FROM
                                    (SELECT
                                        books.title AS title,
                                        books.book_id AS book_id,
                                        ROUND(AVG(ratings.rating), 2) AS avg_rating
                                    FROM
                                        books
                                    INNER JOIN ratings ON ratings.book_id = books.book_id 
                                    GROUP BY books.book_id) AS SUBQ0
                                    LEFT JOIN
                                    (SELECT
                                        books.book_id AS book_id,
                                        COUNT(reviews.review_id) AS cnt
                                    FROM
                                        books
                                    INNER JOIN reviews ON reviews.book_id = books.book_id
                                    GROUP BY books.book_id) AS SUBQ1
                                    ON SUBQ0.book_id = SUBQ1.book_id
                                 ORDER BY
                                    SUBQ0.avg_rating DESC
                               '''
result(count_reviews_and_avg_rating)

,title,avg_rating,cnt
0,Pop Goes the Weasel (Alex Cross #5),5.00,2.0
1,The Ghost Map: The Story of London's Most Terr...,5.00,2.0
2,In the Hand of the Goddess (Song of the Liones...,5.00,2.0
3,Tai-Pan (Asian Saga #2),5.00,2.0
4,How to Be a Domestic Goddess: Baking and the A...,5.00,1.0
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,Junky,2.00,2.0
997,His Excellency: George Washington,2.00,2.0
998,Drowning Ruth,2.00,3.0


По обзорам лидируют Сумерки, а отличников у нас достаточно.

* Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [11]:
top_publisher = '''
                    SELECT
                        publishers.publisher AS publisher,
                        COUNT(publisher) AS cnt
                    FROM
                        books
                    INNER JOIN publishers ON publishers.publisher_id = books.publisher_id    
                    WHERE
                        num_pages > 50
                    GROUP BY
                        publisher
                    ORDER BY cnt DESC
                    LIMIT 1
                 '''
result(top_publisher)

,publisher,cnt
0,Penguin Books,42


* Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [12]:
top_authors =                    '''
                                 SELECT
                                     SUBQ0.cnt_rating,
                                     SUBQ0.avg_rating,
                                     SUBQ0.title,
                                     SUBQ1.author
                                 FROM
                                    (SELECT
                                        books.title AS title,
                                        books.author_id AS author_id,
                                        COUNT(rating) as cnt_rating,
                                        ROUND(AVG(ratings.rating), 2) AS avg_rating
                                    FROM
                                        books
                                    INNER JOIN ratings ON ratings.book_id = books.book_id 
                                    GROUP BY books.book_id
                                    HAVING COUNT(rating) >= 50) AS SUBQ0
                                    LEFT JOIN
                                    (SELECT
                                        authors.author as author,
                                        authors.author_id as author_id
                                    FROM
                                        authors) AS SUBQ1
                                    ON SUBQ0.author_id = SUBQ1.author_id
                                 ORDER BY
                                    SUBQ0.avg_rating DESC
                                 '''
result(top_authors)

,cnt_rating,avg_rating,title,author
0,82,4.41,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré
1,74,4.39,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien
2,80,4.29,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré
3,53,4.26,The Book Thief,Markus Zusak/Cao Xuân Việt Khương
4,73,4.25,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré
5,75,4.19,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré
6,52,4.19,Little Women,Louisa May Alcott
7,88,4.13,The Hobbit or There and Back Again,J.R.R. Tolkien
8,62,4.08,The Lightning Thief (Percy Jackson and the Oly...,Rick Riordan
9,71,3.90,Lord of the Flies,William Golding


Издательство, выпустившее больше всего книг - Penguin Books. Автор с самой высокой средней оценкой - Дж. Роулинг. 

* Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок;

In [13]:
avg_reviews = '''
                SELECT
                    ROUND(AVG(SUBQ2.cnt_reviews), 2) AS avg_reviews
                FROM
                    (SELECT
                        username,
                        COUNT(username) AS cnt_rating
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(username) > 50) AS SUBQ
                    LEFT JOIN
                    (SELECT
                        username,
                        COUNT(username) AS cnt_reviews
                    FROM
                        reviews
                    GROUP BY username) AS SUBQ2
                    ON SUBQ.username = SUBQ2.username
              '''
result(avg_reviews)

,avg_reviews
0,24.33


В среднем кол-во обзоров от юзеров поставивших больше 50 оценок равно 24.33